# Project

In [1]:
using CSV, DataFrames, Random, Gurobi, JuMP, Statistics, ScikitLearn

### Data

In [7]:
data =  CSV.read("data.csv", DataFrame);
all_costs = CSV.read("LCOE_Australia.csv", DataFrame);
all_emissions = CSV.read("energy_emissions.csv", DataFrame);

In [8]:
first(data,5)

Row,date,demand,RRP,demand_pos_RRP,RRP_positive,demand_neg_RRP,RRP_negative,frac_at_neg_RRP,min_temperature,max_temperature,solar_exposure,rainfall,school_day,holiday
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64?,String1,String1
1,2015-01-01,99635.0,25.6337,97319.2,26.416,2315.79,-7.24,0.0208333,13.3,26.9,23.6,0.0,N,Y
2,2015-01-02,129606.0,33.139,121082.0,38.8377,8523.99,-47.8098,0.0625,15.4,38.8,26.8,0.0,N,N
3,2015-01-03,1.42301e5,34.5649,1.42301e5,34.5649,0.0,0.0,0.0,20.0,38.2,26.5,0.0,N,N
4,2015-01-04,1.04331e5,25.0056,1.04331e5,25.0056,0.0,0.0,0.0,16.3,21.4,25.2,4.2,N,N
5,2015-01-05,1.18132e5,26.7242,1.18132e5,26.7242,0.0,0.0,0.0,15.0,22.0,30.7,0.0,N,N


In [9]:
# Grouping by 'Plant category' and calculating the mean of 'LCOE (USD/MWh)'
costs = combine(groupby(all_costs, "Plant category"), "LCOE (USD/MWh)" => mean)

# Renaming the aggregated column for clarity
rename!(costs, "LCOE (USD/MWh)_mean" => :Average_LCOE)

Row,Plant category,Average_LCOE
,String7,Float64
1,Solar,84.1
2,Wind,64.18
3,Gas,111.957
4,Coal,101.88
5,Lignite,104.425


In [10]:
emissions = [0.86, 0.86, 2.38, 2.3, 0.97];
cost = costs[:, :Average_LCOE];
n_energies = 1:5;

In [11]:
# Note: RRP = recommended retail price

# Drop useless columns
select!(data, Not([:demand_pos_RRP, :RRP_positive, :demand_neg_RRP, :RRP_negative, :frac_at_neg_RRP]));

# Transform Yes / No data into binary 
data.holiday .= replace.(data.holiday, "Y" => 1, "N" => 0);
data.school_day .= replace.(data.school_day, "Y" => 1, "N" => 0);
data[!,:holiday] = [parse(Float64, x) for x in data[!,:holiday]];
data[!,:school_day] = [parse(Float64, x) for x in data[!,:school_day]];

# Drop missing values
data = dropmissing(data);

first(data, 4)

Row,date,demand,RRP,min_temperature,max_temperature,solar_exposure,rainfall,school_day,holiday
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2015-01-01,99635.0,25.6337,13.3,26.9,23.6,0.0,0.0,1.0
2,2015-01-02,129606.0,33.139,15.4,38.8,26.8,0.0,0.0,0.0
3,2015-01-03,1.42301e5,34.5649,20.0,38.2,26.5,0.0,0.0,0.0
4,2015-01-04,1.04331e5,25.0056,16.3,21.4,25.2,4.2,0.0,0.0


In [13]:
# Create X, y

X_train = data[1:size(data,1)-36,:]; # until 2020-09
select!(X_train, Not([:demand, :date]));
y_train = data[1:size(data,1)-36,:demand]; # 2070 rows

X_test = data[size(data,1)-35:size(data,1),:]; # 2020-09
select!(X_test, Not([:demand, :date]));
y_test = data[size(data,1)-35:size(data,1),:demand]; # 36 rows

### Random Forest

In [14]:
# Train Random Forest using IAI using the training set
rf_naive = IAI.RandomForestRegressor(random_seed=1)
grid_naive = IAI.GridSearch(rf_naive, max_depth=1:10, num_trees=[5,15,25,30,50,100,150], criterion=[:mse])
IAI.fit!(grid_naive, X_train, y_train, validation_criterion = :mse)
rf_naive = IAI.get_learner(grid_naive)

[ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
┌ Warning: Interpretable AI license expires soon: 2023-12-31T00:00:00. If you need to renew, please send us the following machine ID:
└ 4042a207d88750d8923eca9d55e9b02fe46e2e42c91e28a0e4f395484d554b6c


Fitted RandomForestRegressor

In [15]:
# Get predictions on the test set
rf_naive_pred_test = IAI.predict(rf_naive, X_test);

In [18]:
accuracy_naive = IAI.score(rf_naive, X_test, y_test)

-0.5364321250629525

In [87]:
max_capacity = 1000000
max_emissions = 100000

model = Model(Gurobi.Optimizer)

@variable(model, z1[i=1:size(X_test,1), k=n_energies]) # produced 
@variable(model, z2[i=1:size(X_test,1), k=n_energies]) # purchased 

#@objective(model, Min, (sum(sum(z1[i,k] * cost[i,k] for k=n_energies + z2[i,k] * sold_at[i,k]) - rf_pred_test[i] * X_test[i,:RRP] for i=1:size(X_test,1))))
@objective(model, Min, (sum(sum(z1[i,k] * cost[k] for k=n_energies) - rf_pred_test[i] * X_test[i,:RRP] for i=1:size(X_test,1))))

@constraint(model, demand_constraint[i=1:size(X_test,1)], rf_pred_test[i] <= sum(z1[i,k] + z2[i,k] for k=n_energies))
@constraint(model, capacity_constraint[i=1:size(X_test,1)], max_capacity <= sum(z1[i,k] for k=n_energies))
@constraint(model, emissions_constraint[i=1:size(X_test,1)], sum(z1[i,k] * emissions[k] for k=n_energies) <= max_emissions)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22


36-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 emissions_constraint[1] : 0.86 z1[1,1] + 0.86 z1[1,2] + 2.38 z1[1,3] + 2.3 z1[1,4] + 0.97 z1[1,5] ≤ 100000
 emissions_constraint[2] : 0.86 z1[2,1] + 0.86 z1[2,2] + 2.38 z1[2,3] + 2.3 z1[2,4] + 0.97 z1[2,5] ≤ 100000
 emissions_constraint[3] : 0.86 z1[3,1] + 0.86 z1[3,2] + 2.38 z1[3,3] + 2.3 z1[3,4] + 0.97 z1[3,5] ≤ 100000
 emissions_constraint[4] : 0.86 z1[4,1] + 0.86 z1[4,2] + 2.38 z1[4,3] + 2.3 z1[4,4] + 0.97 z1[4,5] ≤ 100000
 emissions_constraint[5] : 0.86 z1[5,1] + 0.86 z1[5,2] + 2.38 z1[5,3] + 2.3 z1[5,4] + 0.97 z1[5,5] ≤ 100000
 emissions_constraint[6] : 0.86 z1[6,1] + 0.86 z1[6,2] + 2.38 z1[6,3] + 2.3 z1[6,4] + 0.97 z1[6,5] ≤ 100000
 emissions_constraint[7] : 0.86 z1[7,1] + 0.86 z1[7,2] + 2.38 z1[7,3] + 2.3 z1[7,4] + 0.97 z1[7,5] ≤ 100000
 emissions_constraint[8] : 0.86 z1[8,1] + 0.86 z1[8,2] + 2.38 z1[8,3] + 

In [88]:
optimize!(model);

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 108 rows, 360 columns and 720 nonzeros
Model fingerprint: 0xab008b63
Coefficient statistics:
  Matrix range     [9e-01, 2e+00]
  Objective range  [6e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+05, 1e+06]
Presolve removed 36 rows and 180 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible or unbounded model

User-callback calls 27, time in user-callback 0.01 sec


In [38]:
y_test

36-element Vector{Float64}:
 119645.18
 113547.67
 109327.46
 114865.645
 105534.01
  95865.14
  99778.2
 112421.09
 112140.47000000002
 109328.33500000004
 106643.05000000002
 111644.60000000002
  96225.90999999996
      ⋮
 126354.67999999998
 106694.96499999998
 101703.49
 114651.13999999994
 112076.46
 113620.21000000002
 106641.79
  99585.83499999998
  92277.025
  94081.56499999996
 113610.03000000006
 122607.56